In [1]:
import grpc

import server_pb2
import server_pb2_grpc
import numpy as np

In [2]:
# Create a channel
channel = grpc.insecure_channel('andromeda.fi.muni.cz:58110')

In [3]:
# Create a stub (client)
stub = server_pb2_grpc.HomeworkStub(channel)

In [4]:
# Create a request with university email
request = server_pb2.BeginData(email='555179@mail.muni.cz')

In [5]:
# Make the call to the Begin method
response = stub.Begin(request)
session_token = response.token

In [6]:
response

token: "93d5f9fe-8f95-429f-8a9b-f76ccf65e1be"
info: "Define the following two messages:\n    message AdderTask {\n        int64 a = 1;\n        float b = 2;\n    }\n\n    message AdderTaskResponse {\n        string token = 1;\n        double result = 2;\n    }\n\nThe AdderTask in \"oneof tasks\" in the Response message has the field number 8.\n\nExtend the Homework service with the following method:\n    rpc Adder(AdderTaskResponse) returns (Response);\n\nTo get the next task, you have to call the Adder service with the response filled\nwith your assigned token and the result of the sum of the values \"a\" and \"b\" in\nthe AdderTask.\n"
adderTask {
  a: 822
  b: 631.028748
}

In [7]:
# Calculate the sum of a and b
a = response.adderTask.a
b = response.adderTask.b
result = a + b
result

1453.0287475585938

In [8]:
adder_response = server_pb2.AdderTaskResponse(
    token=session_token,
    result=result
)


In [9]:
adder_response

token: "93d5f9fe-8f95-429f-8a9b-f76ccf65e1be"
result: 1453.0287475585938

In [10]:
request2 = server_pb2.AdderTaskResponse(token=adder_response.token, result=adder_response.result)
response2 = stub.Adder(request2)

In [11]:
response2

token: "93d5f9fe-8f95-429f-8a9b-f76ccf65e1be"
info: "Define the following two messages:\n    message MatrixTask {\n        message Row {\n            repeated int64 values = 1; \n        }\n        repeated Row rows = 1;\n    }\n\n    message MatrixTaskResponse {\n        string token = 1;\n        int64 determinant = 2;\n    }\n\nThe MatrixTask in \"oneof tasks\" in the Response message has the field number 9.\n\nExtend the Homework service with the following method:\n    rpc Matrix(MatrixTaskResponse) returns (Response);\n\nTo get the next task, you have to call the Matrix method with the response filled\nwith your assigned token and the matrix determinant.\nSee https://en.wikipedia.org/wiki/Determinant for the description of the matrix determinant."
matrixTask {
  rows {
    values: 13
    values: 17
    values: 19
  }
  rows {
    values: 12
    values: 16
    values: 17
  }
  rows {
    values: 15
    values: 19
    values: 16
  }
}

In [12]:
# Calculate determinant of matrix
matrix = []
for row in response2.matrixTask.rows:
    matrix_row = list(row.values)
    matrix.append(matrix_row)
n_array = np.array(matrix)
det = np.linalg.det(n_array)
float(det)

-28.000000000000036

In [13]:
matrix_response = server_pb2.MatrixTaskResponse(
    token=response2.token,
    determinant=int(det)
)

In [14]:
matrix_response

token: "93d5f9fe-8f95-429f-8a9b-f76ccf65e1be"
determinant: -28

In [15]:
request3 = server_pb2.MatrixTaskResponse(token=matrix_response.token, determinant=matrix_response.determinant)
response3 = stub.Matrix(request3)

In [16]:
response3

token: "93d5f9fe-8f95-429f-8a9b-f76ccf65e1be"
info: "Define the following three messages:\n    message FlipLineRequest {\n        string token = 1;\n    }\n\n    message FlipLineResponse {\n        optional string token = 1;\n        Point point = 2;\n    }\n\n    message Point {\n        sint32 x = 1;\n        sint32 y = 2;\n    }\n\nExtend the Homework service with the following method:\n    rpc ReadLine(FlipLineRequest) returns (stream Point);\n    rpc SendLine(stream FlipLineResponse) returns (Response);\n\nTo get the next task, you have to call the call the ReadLine method (with your assigned token),\nread the points and send them back (SendLine) to the server wrapped in FlipLineResponse with\nreversed X and Y axis. Note that at least one response needs to have your assigned token."

In [17]:
request4 = server_pb2.FlipLineRequest(token=response3.token)
response4 = stub.ReadLine(request4)

In [18]:
response4

<_MultiThreadedRendezvous object>

In [19]:
flipped_points = []
for point in response4:
        print(f"Received point: ({point.x}, {point.y})")
        flipped_point = server_pb2.Point(x=point.y, y=point.x)
        print(flipped_point)
        flipped_points.append(flipped_point)

Received point: (-1, -2)
x: -2
y: -1

Received point: (-2, -2)
x: -2
y: -2

Received point: (0, 0)

Received point: (4, 4)
x: 4
y: 4

Received point: (1, -4)
x: -4
y: 1

Received point: (-3, 0)
y: -3



In [20]:
flipped_points

[x: -2
 y: -1,
 x: -2
 y: -2,
 ,
 x: 4
 y: 4,
 x: -4
 y: 1,
 y: -3]

In [21]:
def generate_responses():
    for point in flipped_points:
        yield server_pb2.FlipLineResponse(token=response3.token, point=point)
response5 = stub.SendLine(generate_responses())

In [22]:
response5

token: "93d5f9fe-8f95-429f-8a9b-f76ccf65e1be"
info: "You have successfully completed your homework.\nSubmit your source code to the IS depository.\n"